# LSSTCam pointing errors
# Here we take three separate Tpoint files, subtract off the solved for IA and IE values, and combine to a new file.

Craig Lage - 18-Apr-25

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def getTpointFile(filename):
    data = np.loadtxt(filename, skiprows=5, usecols=(0,1,2,3,4), comments='END')
    df = pd.DataFrame(data, columns = ['trueAzs', 'trueEls', 'azs', 'els', 'rots'])
    df['deltaAzs'] = (df['azs'] - df['trueAzs']) * 3600
    df['deltaEls'] = (df['els'] - df['trueEls']) * 3600
    #df['deltaAzs'] = (df['trueAzs'] - df['azs']) * 3600
    #df['deltaEls'] = (df['trueEls'] - df['els']) * 3600
    azs = df['azs'].values
    trueAzs = df['trueAzs'].values
    els = df['els'].values
    trueEls = df['trueEls'].values
    deltaAzs = df['deltaAzs'].values
    deltaEls = df['deltaEls'].values
    rots = df['rots'].values
    return azs, els, trueAzs, trueEls, rots, deltaAzs, deltaEls

In [ ]:
files = [[20250531, 0], [20250601, 0], [20250604, 0]]
offsets = [[156.0, -62.0], [113.0, -36.0], [160.0, -74.0]]

outfilename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Input_Combined_18Jun25.dat"
outfile = open(outfilename, 'w')
outfile.write("!" + outfilename + "\n")
outfile.write("!Simonyi Telescope file,June 18, 2025 \n")
outfile.write(": ALTAZ\n")
outfile.write(": ROTNR\n")
outfile.write("-30 14 40.2\n")

for n, [dayObs, stateChanges] in enumerate(files):
    filename = f"/home/c/cslage/u/LSSTCam/data/Tpoint_Input_{dayObs}_{stateChanges}.dat"
    azs, els, trueAzs, trueEls, rots, deltaAzs, deltaEls = getTpointFile(filename)
    IA, IE = offsets[n]
    azs -= IA / 3600.0
    els += IE / 3600.0
    for i in range(len(els)):
        outfile.write(f"{trueAzs[i]:.9f}\t{trueEls[i]:.6f}\t{azs[i]:.9f}\t{els[i]:.9f}\t{rots[i]:.6f}\t\n")
outfile.write("END\n")
outfile.close()


In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.scatter(allAzsRad, allEls, marker = 'x')
ax.set_rmax(60.0)
r_values = [10.0, 20.0, 30.0, 40.0, 50.0]
r_labels = [80.0, 70.0, 60.0, 50.0, 40.0]
ax.set_rgrids(r_values, r_labels)
ax.grid(True)

files = [[20250531, 0], [20250601, 0], [20250604, 0]]
for n, [dayObs, stateChanges] in enumerate(files):
    filename = f"/home/c/cslage/u/LSSTCam/data/Tpoint_Input_{dayObs}_{stateChanges}.dat"
    azs, els, trueAzs, trueEls, rots, deltaAzs, deltaEls = getTpointFile(filename)
    ax.scatter(azs * np.pi / 180.0, 90.0 - els, label=dayObs, marker = 'x')
ax.legend()
plt.savefig("/home/c/cslage/u/LSSTCam/data/Sky_Coverage_20250531_0601_0604.png")